In [ ]:
import SongStructure
import importlib
from Laplacian import *
import numpy as np
import matplotlib.pyplot as plt
import argparse
import sys

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--filename', type=str, required=True, help="Path to audio file")
parser.add_argument('--do_animation', type=int, default=0, help='Save plots of iterations to disk')
parser.add_argument('--plot_result', type=int, default=1, help='Plot the result of fusion')
parser.add_argument('--sr', type=int, default=22050, help='Sample rate to use')
parser.add_argument('--hop_length', type=int, default=512, help="Hop Size in samples")
parser.add_argument('--win_fac', type=int, default=10, help="Number of windows to average in a frame.  If negative, then do beat tracking, and subdivide by |win_fac| times within each beat")
parser.add_argument('--wins_per_block', type=int, default=20, help="Number of frames to stack in sliding window for every feature")
parser.add_argument('--K', type=int, default=5, help="Number of nearest neighbors in similarity network fusion.  If -1, then autotune to sqrt(N) for an NxN similarity matrix")
parser.add_argument('--reg_diag', type=float, default=1.0, help="Regularization for self-similarity promotion")
parser.add_argument('--reg_neighbs', type=float, default=0.5, help="Regularization for direct neighbor similarity promotion")
parser.add_argument('--niters', type=int, default=3, help="Number of iterations in similarity network fusion")
parser.add_argument('--neigs', type=int, default=8, help="Number of eigenvectors in the graph Laplacian")
parser.add_argument('--matfilename', type=str, default="out.mat", help="Name of the .mat file to which to save the results")
parser.add_argument('--jsonfilename', type=str, default="out.json", help="Name of the .json file to which to save results for viewing in the GUI")
parser.add_argument('--diffusion_znormalize', type=int, default=1, help="Whether to perform Z-normalization with diffusion maps to spread things out more")

sys.argv = ["SongStructure.py", "--filename", "MJ.mp3", "--plot_result", "1"]
opt = parser.parse_args()

res = getFusedSimilarity(opt.filename, sr=opt.sr, \
    hop_length=opt.hop_length, win_fac=opt.win_fac, wins_per_block=opt.wins_per_block, \
    K=opt.K, reg_diag=opt.reg_diag, reg_neighbs=opt.reg_neighbs, niters=opt.niters, \
    do_animation=opt.do_animation, plot_result=opt.plot_result, do_crema=True)

In [ ]:
logfloor_quantile = 0.01
Ws = res['Ws']
features = res['features']
times = res['times']
timeticks = np.arange(times[0], times[-1], 50)
timeticks_str = ['%i:%.2i'%(int(t/60), t%60) for t in timeticks]
for f in Ws:
    D = np.array(Ws[f])
    floor = np.quantile(D.flatten(), logfloor_quantile)
    D = np.log(D + floor)
    plt.figure(figsize=(18, 6))
    if 'Fused' not in f:
        plt.subplot2grid((1, 10), (0, 0), colspan=6)
        x = features[f]['x']
        plt.title(f)
        y_axis = ''
        if 'CREMA' in f or 'Chroma' in f:
            y_axis = 'chroma'
        elif 'Tempo' in f:
            y_axis = 'tempo'
        if len(y_axis) > 0:
            librosa.display.specshow(x, x_axis='time', y_axis=y_axis)
        else:
            librosa.display.specshow(x, x_axis='time')
    plt.subplot2grid((1, 10), (0, 6), colspan=4)
    plt.imshow(D, cmap ='magma_r', extent=(times[0], times[-1], times[-1], times[0]), interpolation='nearest')
    
    plt.xticks(timeticks, timeticks_str)
    plt.yticks(timeticks, timeticks_str)
    plt.title("%s Similarity Matrix"%f)
    if 'Fused' not in f:
        plt.savefig("MJ_%s.svg"%f, bbox_inches='tight')

In [ ]:
# Now vary window length
for wins_per_block in range(1, 21):
    res = getFusedSimilarity(opt.filename, sr=opt.sr, \
        hop_length=opt.hop_length, win_fac=opt.win_fac, wins_per_block=wins_per_block, \
        K=opt.K, reg_diag=opt.reg_diag, reg_neighbs=opt.reg_neighbs, niters=opt.niters, \
        do_animation=opt.do_animation, plot_result=opt.plot_result, do_crema=True)
    plotFusionResults(res["Ws"], {}, {}, res["times"], win_fac=opt.win_fac, wins_per_block = wins_per_block)
    plt.savefig("Wins%i.png"%wins_per_block, bbox_inches='tight')

In [ ]:
lapfn = getRandomWalkLaplacianEigsDense
specfn = lambda v, dim, times: spectralClusterSequential(v, dim, times, rownorm=False)
sr=22050
hop_length=512
win_fac=10
wins_per_block=20
K=3
reg_diag=1.0
reg_neighbs=0.0
niters=10
neigs=10

Ws = res["Ws"]
times = res["times"]
vs = {name:lapfn(Ws[name])[:, 1:neigs+1] for name in Ws}
alllabels = {name:[specfn(vs[name], k, times) for k in range(2, neigs+1)] for name in Ws}
#print("Elapsed time spectral clustering: %.3g"%(time.time()-tic))
specintervals_hier = {name:[res['intervals_hier'] for res in alllabels[name]] for name in alllabels}
speclabels_hier = {name:[res['labels_hier'] for res in alllabels[name]] for name in alllabels}


In [ ]:
importlib.reload(SongStructure)
#FeatureOrder = ['Chromas', 'MFCCs', 'Fused MFCC/Chroma', 'CREMA', 'Tempogram', 'Fused']
FeatureOrder = ['Chromas']
for l in range(8):
    levels = [l]
    SongStructure.plotFusionResults(Ws, vs, alllabels, times, opt.win_fac, FeatureOrder=FeatureOrder, levels=levels)#, specintervals_hier, speclabels_hier)
    plt.savefig("ClusterLevel%i.png"%l, bbox_inches='tight')

In [ ]:
y, sr = librosa.load("MJ.mp3", sr=22050)

In [ ]:
# Segment out song based on clusters from fused Chroma/MFCC
ls = alllabels['Fused MFCC/Chroma']
ls = [l['labels'] for l in [ls[3], ls[6]]]
print(ls[0].shape)
print(sr)

In [ ]:
from scipy.io import wavfile
plt.figure(figsize=(12, 2))
for i, l in enumerate(ls):
    l = np.array(l) + 1
    l[0] = np.max(l)+1
    plt.clf()
    plt.imshow(l[None, :], cmap='gray', aspect='auto')
    plt.axis('off')
    plt.savefig("Division%i.png"%i, bbox_inches='tight')
    for k in np.unique(l):
        i1 = np.argmax(l == k)
        i2 = np.argmin(l[i1::] == k) + i1
        i1 *= 5120
        i2 *= 5120
        print(i, i1, i2)
        x = y[i1:i2]
        wavfile.write("MJ_%i_%i.wav"%(i, k), sr, x)
        
        plt.clf()
        plt.imshow(l[None, :], cmap='magma_r', aspect='auto')
        plt.axis('off')
        lk = np.array(l)
        lk[np.abs(lk-k) > 0] = 0
        lk[lk > 0] = 255
        lk[lk == 0] = 0
        lk = 255-lk
        lk = np.array(lk[None, :, None], dtype=int)
        lk = np.concatenate((lk, lk, lk, lk), axis=2)
        plt.imshow(lk, aspect='auto')
        plt.axis('off')
        plt.savefig("Division%i_%i.png"%(i, k), bbox_inches='tight')